In [4]:
import pandas as pd
import numpy as np

In [5]:
def matriz(num_colunas, array1):
    """
    Gera uma matriz sequencial a partir de um array, com o número de colunas especificado.

    Args:
        array (list ou np.ndarray): Array de entrada.
        num_colunas (int): Número de colunas desejado na matriz.

    Returns:
        np.ndarray: Matriz sequencial.
    """
    if num_colunas > len(array1):
        raise ValueError("O número de colunas não pode ser maior que o tamanho do array.")

    # Número de linhas na matriz
    num_linhas = len(array1) - num_colunas + 1

    # Criando a matriz sequencial
    matriz = np.array([array1[i:i + num_colunas] for i in range(num_linhas)])
    return matriz

In [6]:
array = np.arange(300)

array

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [7]:
matriz1 = matriz(30,array)
matriz1

array([[  0,   1,   2, ...,  27,  28,  29],
       [  1,   2,   3, ...,  28,  29,  30],
       [  2,   3,   4, ...,  29,  30,  31],
       ...,
       [268, 269, 270, ..., 295, 296, 297],
       [269, 270, 271, ..., 296, 297, 298],
       [270, 271, 272, ..., 297, 298, 299]])

In [12]:
def media_movel(array, janela):
    return np.convolve(array, np.ones(janela)/janela, mode='valid')

# Gerar a matriz original
X_original = matriz(30, array)[:, :-1]  # Pegamos as primeiras 29 colunas

# Criar a matriz de médias móveis
media = media_movel(array, janela=5)  # Exemplo: média de 5 períodos
X_media = matriz(30, media)[:, :-1]  # Ajustando para o mesmo formato

# Concatenar as duas matrizes como uma única entrada
X_combinado = np.hstack([X_original, X_media])
y = matriz(30, array)[:, -1]  # Última coluna como saída


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 271 and the array at index 1 has size 267

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

# Definir camadas de entrada
input1 = Input(shape=(X_original.shape[1],))
input2 = Input(shape=(X_media.shape[1],))

# Criar caminhos separados para processar cada matriz
x1 = Dense(64, activation="relu")(input1)
x2 = Dense(64, activation="relu")(input2)

# Concatenar os dois caminhos
concat = Concatenate()([x1, x2])
output = Dense(1, activation="linear")(concat)

# Criar o modelo com múltiplas entradas
modelo = Model(inputs=[input1, input2], outputs=output)
modelo.compile(optimizer="adam", loss="mse")

# Treinar o modelo
modelo.fit([X_original, X_media], y, epochs=50, batch_size=32)


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 271, 267
'y' sizes: 271


In [13]:
def media_movel(array, janela):
    return np.convolve(array, np.ones(janela)/janela, mode='valid')

# Gerar a matriz original
X_original = matriz(30, array)[:, :-1]  # Pegamos as primeiras 29 colunas

# Criar a matriz de médias móveis
media = media_movel(array, janela=5)  # Média móvel de janela 5
X_media = matriz(30, media)[:, :-1]   # Ajustando para mesmo formato

# Ajustar as dimensões para garantir que todas tenham o mesmo número de linhas
min_linhas = min(X_original.shape[0], X_media.shape[0])
X_original = X_original[:min_linhas]
X_media = X_media[:min_linhas]
y = matriz(30, array)[:, -1][:min_linhas]  # Ajustar saída

# Concatenar para um único modelo (se necessário)
X_combinado = np.hstack([X_original, X_media])

print("Dimensões ajustadas:")
print("X_original:", X_original.shape)
print("X_media:", X_media.shape)
print("y:", y.shape)


Dimensões ajustadas:
X_original: (267, 29)
X_media: (267, 29)
y: (267,)


In [14]:
X_original

array([[  0,   1,   2, ...,  26,  27,  28],
       [  1,   2,   3, ...,  27,  28,  29],
       [  2,   3,   4, ...,  28,  29,  30],
       ...,
       [264, 265, 266, ..., 290, 291, 292],
       [265, 266, 267, ..., 291, 292, 293],
       [266, 267, 268, ..., 292, 293, 294]])

In [15]:
X_media

array([[  2.,   3.,   4., ...,  28.,  29.,  30.],
       [  3.,   4.,   5., ...,  29.,  30.,  31.],
       [  4.,   5.,   6., ...,  30.,  31.,  32.],
       ...,
       [266., 267., 268., ..., 292., 293., 294.],
       [267., 268., 269., ..., 293., 294., 295.],
       [268., 269., 270., ..., 294., 295., 296.]])